In [1]:
import pandas as pd
pd.set_option('display.max_columns', 30)
import ScraperFC as sfc
import pkgutil
for sub in pkgutil.iter_modules(sfc.__path__):
    print(sub.name)
# from ScraperFC import FBRef

FBRef
FiveThirtyEight
ScraperFC
Understat
shared_functions


In [ ]:
try:
#     scraper = sfc.FBRef()
#     lg = 'EPL'
#     s,v = scraper.scrape_adv_gk(2020,lg,False)

#     scraper = sfc.Understat()
#     v = scraper.scrape_matches(2019,False)

    scraper = sfc.ScraperFC()
    v = scraper.scrape_matches(2019,False)

    scraper.close()
except e:
    print(e)
    scraper.close()

Scraping match 211/380 in the 2018-2019 EPL season.


In [ ]:
v